In [1]:
import urllib.request

# Download dataset helpdesk TIK UB
url = 'https://docs.google.com/spreadsheets/d/1PzUlTZwY6IySZ7VNotDIH3h9hnuRIuy40CgAS7BcFkE/export?gid=1874021283&format=csv'
output_file = 'test.csv'

urllib.request.urlretrieve(url, output_file)

print(f"File '{output_file}' berhasil diunduh.")

File 'test.csv' berhasil diunduh.


In [2]:
from datasets import Dataset, DatasetDict
from bs4 import BeautifulSoup
import pandas as pd
import re

/home/fatahap27/.conda/envs/NER-Project/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dataset_target_domain = pd.read_csv('test.csv', usecols=['body'])

In [4]:
len(dataset_target_domain['body'])

14605

In [5]:
dataset_target_domain['body'] = dataset_target_domain['body']
dataset_target_domain = dataset_target_domain.drop_duplicates(subset='body')
dataset_target_domain = dataset_target_domain.reset_index(drop=True)
# dataset_target_domain['body'] = dataset_target_domain['body'].apply(list)
len(dataset_target_domain['body'])

9619

In [6]:
def remove_html_tag(x):
    if x is not None:
        # Menghilangkan tag blockquote beserta isinya
        text_no_blockquote = re.sub(r'<blockquote>.*?</blockquote>', '', x, flags=re.DOTALL)
        
        # Menghilangkan tag HTML
        text_no_html = re.sub(r'<[^>]+>', '', text_no_blockquote)
        
        return text_no_html
    return None

def remove_blockquote_tag(sentence):
    if sentence is not None:
        return re.sub(r'<blockquote>.*?</blockquote>', '', sentence, flags=re.DOTALL)
    return None

def remove_html_tag(x):
    if x is not None:
        return re.sub(r'<[^>]+>', '', x)
    return None

dataset_target_domain['body'] = dataset_target_domain['body'].apply(remove_blockquote_tag)
dataset_target_domain['body'] = dataset_target_domain['body'].apply(remove_html_tag)

In [7]:
from transformers.generation import TFGenerationMixin

2023-12-20 12:25:34.992539: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-20 12:25:34.992626: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-20 12:25:34.993790: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-20 12:25:35.001172: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-20 12:25:36.068927: W tensorflow/compiler/tf2

In [8]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, AutoModelForSequenceClassification
import torch

In [9]:
tokenizer = AutoTokenizer.from_pretrained("bryanahusna/my-nergrit-model")
model = AutoModelForTokenClassification.from_pretrained("bryanahusna/my-nergrit-model")

In [68]:
tes = ["berbahasa Indonesia, pada tanggal 27 September 2022 09:46", "ini kalimat kedua"]

tes2 = tokenizer(tes, padding=True, return_tensors='pt')
token = tokenizer.tokenize(tes[0])

attention_mask = tes2['attention_mask']

output = model(**tes2)
logits = output.logits

predicted = torch.argmax(logits, dim=2)
predicted = predicted_tes.masked_fill(attention_mask==0, -1)

print(token)
print(len(token))
print()

print(tes2['attention_mask'])
print()

print(predicted)
print(predicted.size())
print()

print(tes2)

['berbahasa', 'indonesia', ',', 'pada', 'tanggal', '27', 'september', '202', '##2', '09', ':', '46']
12

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

tensor([[38, 38,  4, 38, 38, 38,  1, 20, 20, 20, 17, 36, 36, 38],
        [38, 38, 38, 10, 38, -1, -1, -1, -1, -1, -1, -1, -1, -1]])
torch.Size([2, 14])

{'input_ids': tensor([[    3,  6196,  1718,    16,  1560,  2307,  3402,  3264, 24674,   952,
          8336,    30,  6631,     4],
        [    3,  1540,  5859,  2048,     4,     0,     0,     0,     0,     0,
             0,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}
